# RunnableBranch

`RunnableBranch` 는 입력에 따라 동적으로 로직을 라우팅할 수 있는 강력한 도구입니다. 이를 통해 개발자는 **입력 데이터의 특성에 기반하여 다양한 처리 경로를 유연하게 정의** 할 수 있습니다.

`RunnableBranch` 는 복잡한 의사 결정 트리를 간단하고 직관적인 방식으로 구현할 수 있도록 도와줍니다. 이는 코드의 가독성과 유지보수성을 크게 향상시키며, 로직의 모듈화와 재사용성을 촉진합니다.

또한, `RunnableBranch` 는 런타임에 동적으로 분기 조건을 평가하고 적절한 처리 루틴을 선택할 수 있어, 시스템의 적응력과 확장성을 높여줍니다.

이러한 특징들로 인해 RunnableBranch는 다양한 도메인에서 활용될 수 있으며, 특히 입력 데이터의 다양성과 변동성이 큰 애플리케이션 개발에 매우 유용합니다. `RunnableBranch` 를 효과적으로 활용하면 코드의 복잡성을 줄이고, 시스템의 유연성과 성능을 향상시킬 수 있습니다.


## 입력에 따른 동적 로직 라우팅

LangChain Expression Language에서 라우팅을 수행하는 방법에 대해 다룹니다.

라우팅을 통해 이전 단계의 출력이 다음 단계를 정의하는 비결정적 체인을 생성할 수 있습니다. 라우팅은 LLM과의 상호 작용에 구조와 일관성을 제공하는 데 도움이 됩니다.

라우팅을 수행하는 방법에는 두 가지가 있습니다.

1. `RunnableLambda` 에서 조건부로 실행 가능한 객체를 반환 (권장)
2. `RunnableBranch` 사용

두 가지 방법 모두 첫 번째 단계에서 입력 질문을 `수학`, `과학` 또는 `기타`에 대한 것으로 분류한 다음, 해당 프롬프트 체인으로 라우팅하는 두 단계 시퀀스를 사용하여 설명하겠습니다.


In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 간단한 예시

먼저, 들어오는 질문이 `수학`, `과학`, 또는 `기타` 중 하나로 분류하는 Chain을 생성하겠습니다.


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

# 체인을 생성합니다.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

생성한 chain을 사용하여 질문을 분류합니다.


In [3]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [4]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [5]:
# 질문을 입력하여 체인을 호출합니다.
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

이제 세 개의 하위 체인을 생성해 보겠습니다.


In [6]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | ChatOpenAI(model="gpt-4o-mini")
)

## 사용자 정의 함수 사용하기

LangChain 공식 도큐먼트에서 권장하는 방식이며, 서로 다른 출력 간의 라우팅을 위해 **사용자 정의 함수**를 `RunnableLambda` 로 래핑하여 활용할 수도 있습니다.




In [7]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain

In [8]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # 경로를 지정하는 함수를 인자로 전달합니다.
        route
    )
    | StrOutputParser()
)

In [9]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 중요한 분야로, 미분과 적분이라는 두 가지 주요 개념으로 나뉩니다. \n\n미분은 함수의 변화율을 분석하는 방법입니다. 예를 들어, 자동차의 속도를 알아보려면 시간에 따라 위치가 어떻게 변하는지를 고려해야 합니다. 이러한 변화의 순간적인 비율이 미분을 통해 구해집니다.\n\n반면, 적분은 미분의 반대 개념으로, 주어진 함수의 값들을 일정 구간에 걸쳐 합산하여 전체를 구하는 과정입니다. 이는 면적, 부피 등을 계산할 때 유용하게 사용됩니다.\n\n미적분은 자연현상, 공학, 경제학 등 여러 분야에서 응용되며, 함수의 행동을 이해하고 예측하는 데 필수적인 도구로 자리잡고 있습니다. 따라서 미적분의 개념을 이해하는 것은 매우 중요합니다.'

In [18]:
print(full_chain.get_graph().draw_ascii())

                    +-------------------------------+                      
                    | Parallel<topic,question>Input |                      
                    +-------------------------------+                      
                             ***           ***                             
                           **                 **                           
                         **                     **                         
              +----------------+                  **                       
              | PromptTemplate |                   *                       
              +----------------+                   *                       
                       *                           *                       
                       *                           *                       
                       *                           *                       
                +------------+                     *                       
            

In [ ]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

In [ ]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

## RunnableBranch

`RunnableBranch`는 입력값에 따라 실행할 조건과 Runnable을 정의할 수 있는 특별한 유형의 `Runnable` 입니다.

다만, 위에서 설명한 사용자 정의 함수로 구현할 수 없는 기능을 제공하지는 않으므로, 사용자 정의 함수를 사용하는 것이 좋습니다.

**문법**

- `RunnableBranch`는 (조건, Runnable) 쌍의 리스트와 기본 Runnable로 초기화됩니다.
- 호출 시 전달된 입력값을 각 조건에 전달하여 분기를 선택합니다.
- True로 평가되는 첫 번째 조건을 선택하고, 해당 조건에 해당하는 Runnable을 입력값과 함께 실행합니다.
- 제공된 조건과 일치하는 것이 없으면 **기본 `Runnable`** 을 실행합니다.

In [19]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [20]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 함수의 변화를 이해하고 다루기 위한 도구입니다. 미적분은 두 가지 주요 가지로 나뉘는데, 먼저 미분은 함수의 변화율, 즉 기울기를 구하는 방법입니다. 이를 통해 우리는 특정 지점에서 함수가 어떻게 변화하는지를 알 수 있습니다.\n\n반면, 적분은 함수의 면적을 구하거나 누적된 양을 나타내는 개념입니다. 이러한 미분과 적분은 서로 밀접하게 연결되어 있으며, 함수의 미분과 적분에 대한 기본 정리(미적분학의 기본 정리)는 이 두 가지 개념이 서로 반대 관계임을 증명해 줍니다.\n\n미적분은 물리학, 공학, 경제학 등 다양한 분야에서 매우 중요한 역할을 하며, 현실 세계의 여러 현상을 모델링하고 분석하는 데 필수적인 도구입니다.'

In [21]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

"아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 지구와 같은 천체의 중력에 의해 가속되는 비율을 나타내며, 일반적으로 'g'로 표기됩니다. 지구에서의 중력 가속도는 약 9.81 m/s²로 알려져 있습니다. 이를 계산하기 위해서는 다음과 같은 공식을 사용할 수 있습니다:\n\n\\[ g = \\frac{G \\cdot M}{r^2} \\]\n\n여기서 \\( G \\)는 만유인력 상수(약 6.674 × 10^-11 N(m/kg)²), \\( M \\)은 천체의 질량, \\( r \\)은 천체의 중심에서 물체까지의 거리입니다. 따라서, 중력 가속도는 천체의 질량과 반지름에 의존하며, 다양한 천체에서 이 값을 계산할 수 있습니다."

In [22]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG( retrieval-Augmented Generation)는 정보 검색과 생성 모델을 결합한 방법으로, 특정 질문에 대한 답변을 생성하기 위해 외부 데이터베이스에서 관련 정보를 검색한 후 이를 활용하여 더욱 정확한 응답을 생성하는 기술입니다. 이를 통해 AI 모델은 더 폭넓고 신뢰성 있는 정보를 바탕으로 대화를 할 수 있습니다.'